In [ ]:
import os
print('CPUs', os.cpu_count())
import sys
import glob

import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.tests.datafiles import CRD, PSF, DCD, DCD2, PDB
from MDAnalysis.analysis import distances
from numpy import savetxt

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from numpy.linalg import norm
import time

###### 1st STEP ######

In [ ]:
def find_magnitude(x, y, z):
    
    import math
    difference_x = x[1] - x[0]
    difference_y = y[1] - y[0]
    difference_z = z[1] - z[0]
    difference_x = difference_x ** 2
    difference_y = difference_y ** 2
    difference_z = difference_z ** 2
    magnitude = math.sqrt(difference_x + difference_y + difference_z)
    return magnitude

In [ ]:
import multiprocessing


TM2 =  list(map(str, list(range(151,164+1)))) 
TM3 =  list(map(str, list(range(203,220+1)))) 
TM5 =  list(map(str, list(range(289,313+1)))) 
TM6 =  list(map(str, list(range(335,354+1)))) 
TM7 =  list(map(str, list(range(391,414+1)))) 
ICL2 = list(map(str, list(range(221,228+1))))
ICL1 = list(map(str, list(range(145,150+1))))

lst_ga = list(map(str, list(range(20,34+1))))   + list(map(str, list(range(190,195+1)))) + \
         list(map(str, list(range(216,220+1)))) + list(map(str, list(range(256,269+1)))) + \
         list(map(str, list(range(309,354+1))))



_list = [[TM2,lst_ga,'TM2_ga'],[TM3,lst_ga,'TM3_ga'],
         [TM5,lst_ga,'TM5_ga'],[TM6,lst_ga,'TM6_ga'],
         [TM7,lst_ga,'TM7_ga'],[ICL2,lst_ga,'ICL2_ga'],
         [ICL1,lst_ga,'ICL1_ga']]


def dist(filtered_list):
    
    trj = mda.Universe('mdmb_clust.pdb','mdmb_clust_10000.dcd')
    
    lst_1  = filtered_list[0]
    lst_2  = filtered_list[1]
    output = filtered_list[2]
    
    dist_2 = []
    for t in trj.trajectory:
        dist_1 = []
        for i in range(len(lst_1)):
       
            d1 = trj.select_atoms('segid C4 and name CA and resid ' + lst_1[i])
      
            for j in range(len(lst_2)):
           
                d2 = trj.select_atoms('segid C2 and name CA and resid ' + lst_2[j])
            
                x0 = d1.positions[0][0]
                y0 = d1.positions[0][1]
                z0 = d1.positions[0][2]
            
                x1 = d2.positions[0][0]
                y1 = d2.positions[0][1]
                z1 = d2.positions[0][2]

            
                x = [x0, x1]
                y = [y0, y1]
                z = [z0, z1]
        
                dist = find_magnitude(x,y,z)
                dist_1.append(find_magnitude(x,y,z))
            
        dist_2.append(dist_1)
    
         
    distt = np.array(dist_2)
    print(distt.shape)

    savetxt(output + '_dist.csv', distt, fmt='%1.5f', delimiter=',')
    print('DONE')
    
if __name__ == '__main__':
    
    p = multiprocessing.Pool(processes = multiprocessing.cpu_count()-3)
    start = time.time()
    
    for filtered_list in _list:
        
        print(filtered_list[0])
        print(filtered_list[1])
        print(filtered_list[2])
        p.apply_async(dist, [filtered_list])
    p.close()
    p.join()
    print("Complete")
    end = time.time()
    print('total time (s)= ' + str(end-start))